# Framework component tests

In [ ]:
import numpy as np
import pandas as pd

import subprocess
import time
import os

import panson as ps
from panson import bundle

from math import pi, exp, log2

In [ ]:
# import logging
# logging.basicConfig(level=logging.DEBUG)

In [ ]:
!xhost +

In [ ]:
# name given to the container
CONTAINER_NAME = 'openface'

# base directory of the container
CONTAINER_BASE_DIR = '/home/openface-build'
# directory with executalbles in the container
CONTAINER_BIN_DIR = os.path.join(CONTAINER_BASE_DIR, 'build/bin')

FILE_DIR = '../media/files'
OUT_DIR = os.path.join(FILE_DIR, 'processed')

CONTAINER_FILE_DIR = os.path.join(CONTAINER_BASE_DIR, 'files')
CONTAINER_OUT_DIR = os.path.join(CONTAINER_FILE_DIR, 'processed')

CONTAINER_EXECUTABLE = os.path.join(CONTAINER_BIN_DIR, 'FeatureExtraction')


def feature_extraction_offline(video_name):
    
    video_path = os.path.join(FILE_DIR, video_name)
    
    # the file must be in FILE_DIR
    if not os.path.isfile(video_path):
        raise FileNotFoundError(video_path)
    
    container_video_path = os.path.join(CONTAINER_FILE_DIR, video_name)
    
    command = [
        'docker', 'exec', CONTAINER_NAME, CONTAINER_EXECUTABLE,
        '-f', container_video_path,
        '-out_dir', CONTAINER_OUT_DIR,
        # features extracted
        '-pose', '-gaze', '-aus',
        # output tracked video
        '-tracked'
    ]
    
    # capture and combine stdout and stderr into one stream and set as text stream
    proc = subprocess.Popen(command,stdout=subprocess.PIPE, stderr=subprocess.STDOUT, universal_newlines=True)
        
    # poll process and show its output
    while True:
        output = proc.stdout.readline()
        
        if output:
            print(output.strip())
            
        if proc.poll() is not None:
            break
    
    return proc

def feature_extraction_online(pipe='files/pipe'):
    
    command = [
        'docker', 'exec', CONTAINER_NAME, CONTAINER_EXECUTABLE,
        '-device', '0', # use default device
        '-pose', '-gaze', '-aus',
        # '-tracked'
        '-of', pipe
    ]
    
    # capture and combine stdout and stderr into one stream and set as text stream
    proc = subprocess.Popen(command, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, universal_newlines=True)    

    print('Starting real-time analysis...')
    print('Open the pipe from the read side to start the feature stream')
    
    return proc

def kill_feature_extraction_online():
    # !docker exec -it openface pkill FeatureExt
    command = ['docker', 'exec', CONTAINER_NAME, 'pkill', 'FeatureExt']
    subprocess.run(command)

def read_openface_csv(csv_path):
    return pd.read_csv(csv_path, sep=r',\s*', engine='python')

In [ ]:
def ffmpeg_convert(in_file, out_file):
    
    # this was the original command that appeared in the notebook
    # ffmpeg -y -i "files/processed/phone.avi" -c:v libx264 -preset slow -crf 22 -pix_fmt yuv420p -c:a libvo_aacenc -b:a 128k "files/phone-processed.mp4"
    command = [
        'ffmpeg', '-y',
        '-i', in_file,
        '-c:v', 'libx264',
        '-crf', '22',
        '-pix_fmt', 'yuv420p',
        '-c:a', 'libvo_aacenc',
        '-b:a', '128k',
        out_file
    ]
    
    proc = subprocess.run(command, capture_output=True)
    
    return proc

def ffmpeg_merge(video_file, audio_file, out_file):
    
    # ffmpeg -i files/phone-processed.mp4 -i score.wav  -c:v copy phone-processed-son.mp4 -y
                
    command = [
        'ffmpeg', '-y',
        '-i', video_file,
        '-i', audio_file,
        '-map', '0:v',
        '-map', '1:a',
        '-c:v', 'copy',
        out_file
    ]
    
    proc = subprocess.run(command, capture_output=True)
    
    return proc

## Setup

### Supercollider

In [ ]:
import sc3nb as scn

In [ ]:
# start scsynth
sc = scn.startup()
# connect scsynth to the system playback
!jack_connect "SuperCollider:out_1" "system:playback_1"
!jack_connect "SuperCollider:out_2" "system:playback_2"

If this does not work, open QJackCtl and link the nodes in the graph manually.

In [ ]:
sc.exit()

In [ ]:
# test sound output
sc.server.blip()

In [ ]:
sc.server.latency = 0.1

Example data

In [ ]:
df = pd.read_csv(os.path.join(OUT_DIR, "phone.csv"), sep=r',\s*', engine='python')
df.head()

### OpenFace Stream

In [ ]:
from panson import streams

FIFO_PATH = os.path.join(FILE_DIR, 'pipe.csv')

openface_stream = streams.CsvFifo('openface', args=(FIFO_PATH,))\
        .add_open_hook(feature_extraction_online)\
        .add_close_hook(kill_feature_extraction_online)\
        .test()

In [ ]:
openface_stream.sample_size, openface_stream.dtype, openface_stream.fps

## Sonifications

In [ ]:
# a (implicit ID allocation)
class AU04ContinuousSonification(ps.Sonification):
    
    # parameters of the sonification
    amp = ps.FloatSliderParameter(0, 1, 0.01)
        
    def init_parameters(self):
        self.amp = 1
    
    @bundle
    def init_server(self):
        scn.SynthDef.load("/home/michele/Desktop/Thesis/tools/sc3nb/src/sc3nb/resources/synthdefs/s2.scsyndef")

    @bundle
    def start(self):
        # lag time is decided based on the frame rate
        self.synth = scn.Synth("s2", {"amp": 0, "lg": 0.03})

    @bundle
    def _process(self, row):  
        self.synth.set(
            # only "max" should be enough (to clip the top part to 0.3)
            "amp", self.amp * scn.linlin(row["AU04_r"], 0, 1, 0, 0.3, "minmax"),
            # map the intensity of the AU in one octave range
            "freq", scn.midicps(scn.linlin(row["AU04_r"], 0, 5, 69, 81))
        )

In [ ]:
class DropBlink(ps.Sonification):
    
    # hysteresis boundaries
    bounds = ps.FloatRangeSliderParameter(0, 5, 0.1)
    
    # drop definition
    drop_def = scn.SynthDef(
        "drop",
        r"""{ | freq=600, dp=1200, amp=0.5, dur=0.1, pan=0 |
            var sig, env, fch;
            fch = XLine.kr(freq, freq+dp, dur);
            sig = SinOsc.ar(fch);
            env = EnvGen.kr(Env.perc(0.001, dur, curve: -4), 1.0, doneAction: 2);
            Out.ar(0, Pan2.ar(sig, pan, env*amp))
        }"""
    )
    
    def init_parameters(self):
        self.bounds = [1, 1.6]        
    
    @bundle
    def init_server(self):
        self.drop_def.add()

    @bundle
    def start(self):
        self.blinking = False
    
    @bundle
    def stop(self):
        # drops die out alone
        pass
    
    @bundle
    def _process(self, row):
        intensity = row["AU45_r"]
        
        if self.blinking:
            if intensity < self.bounds[0]:
                self.blinking = False
        elif intensity > self.bounds[1]:
            self.blinking = True
            scn.Synth("drop")
            

class DoubleDropBlink(DropBlink):
    
    @bundle
    def _process(self, row):
        intensity = row["AU45_r"]
        
        if self.blinking:
            if intensity < self.bounds[0]:
                self.blinking = False
                scn.Synth("drop", {"freq": 900})
        elif intensity > self.bounds[1]:
            self.blinking = True
            scn.Synth("drop")

In [ ]:
AU01_SAMPLE_PATH = "samples/modified/au01.wav"
AU02_SAMPLE_PATH = "samples/modified/au02.wav"
AU04_SAMPLE_PATH = "samples/modified/au04.wav"
AU05_SAMPLE_PATH = "samples/modified/au05.wav"
AU06_SAMPLE_PATH = "samples/modified/au06.wav"
AU07_SAMPLE_PATH = "samples/modified/au07.wav"
AU09_SAMPLE_PATH = "samples/modified/au09.wav"
AU10_SAMPLE_PATH = "samples/modified/au10.wav"
AU12_SAMPLE_PATH = "samples/modified/au12.wav"
AU14_SAMPLE_PATH = "samples/modified/au14.wav"
AU15_SAMPLE_PATH = "samples/modified/au15.wav"
AU17_SAMPLE_PATH = "samples/modified/au17.wav"
AU20_SAMPLE_PATH = "samples/modified/au20.wav"
AU23_SAMPLE_PATH = "samples/modified/au23.wav"
AU25_SAMPLE_PATH = "samples/modified/au25.wav"
AU26_SAMPLE_PATH = "samples/modified/au26.wav"
AU28_SAMPLE_PATH = "samples/modified/au28.wav"

In [ ]:
from collections import namedtuple

class DirectionalPercussive(ps.Sonification):
    
    # parameters of the sonification
    amp = ps.FloatSliderParameter(0, 1, 0.01)
    pan = ps.CheckboxParameter()
    # hysteresis bounds relative to each intensity level
    bounds = ps.FloatRangeSliderParameter(-1, +1)

    playbuf_def = scn.SynthDef(
        "playbuf_bend",
        r"""{| out=0, bufnum=0, rateInitial=1, amp=1, pan=0, breakTime, rateFinal |
            var sig, rate;

            var rateAvg = (rateInitial + rateFinal) / 2;
            var sampleRateAvg = rateAvg * BufSampleRate.kr(bufnum);
            var breakFrame = breakTime * BufSampleRate.kr(bufnum);
            // mono signal: frames = samples
            var remainingFrames = BufSamples.kr(bufnum) - breakFrame;
            // calculate remaining time with dynamic rate
            var remainingTime = remainingFrames / sampleRateAvg;

            rate = EnvGen.kr(
                Env(
                    [rateInitial, rateInitial, rateFinal],
                    [breakTime, remainingTime]
                )
            );
            sig = PlayBuf.ar(1, bufnum, rate*BufRateScale.kr(bufnum), doneAction:2);
            Out.ar(0, Pan2.ar(sig, pan, amp));
        }"""
    )
    
    AuRecord = namedtuple('AuRecord', ['label', 'path', 'break_time', 'rate_up', 'rate_down'])
    
    AUs = {
        1:  AuRecord('AU01_r', AU01_SAMPLE_PATH, 0.2, 1.1, 0.9),
        2:  AuRecord('AU02_r', AU02_SAMPLE_PATH, 0.2, 1.1, 0.9),
        4:  AuRecord('AU04_r', AU04_SAMPLE_PATH, 0.2, 1.1, 0.9),
        5:  AuRecord('AU05_r', AU05_SAMPLE_PATH, 0.05, 1.1, 0.9),
        6:  AuRecord('AU06_r', AU06_SAMPLE_PATH, 0.03, 1.1, 0.9),
        7:  AuRecord('AU07_r', AU07_SAMPLE_PATH, 0.1, 1.1, 0.9),
        9:  AuRecord('AU09_r', AU09_SAMPLE_PATH, 0.2, 1.5, 0.7),
        10: AuRecord('AU10_r', AU10_SAMPLE_PATH, 0.15, 1.2, 0.9),
        12: AuRecord('AU12_r', AU12_SAMPLE_PATH, 0.1, 1.1, 0.9),
        14: AuRecord('AU14_r', AU14_SAMPLE_PATH, 0.1, 1.1, 0.9),
        15: AuRecord('AU15_r', AU15_SAMPLE_PATH, 0.05, 1.1, 0.9),
        17: AuRecord('AU17_r', AU17_SAMPLE_PATH, 0.05, 1.5, 0.8),
        20: AuRecord('AU20_r', AU20_SAMPLE_PATH, 0.1, 1.1, 0.9),
        23: AuRecord('AU23_r', AU23_SAMPLE_PATH, 0.15, 1.25, 0.85),
        25: AuRecord('AU25_r', AU25_SAMPLE_PATH, 0.005, 2, 0.5),
        26: AuRecord('AU26_r', AU26_SAMPLE_PATH, 0.05, 2, 0.5)
    }
    
    def init_parameters(self, pan=False):
        self.amp = 0.3
        self.pan = pan
        self.bounds = [-0.3, +0.3]

    @bundle
    def init_server(self):
        self.playbuf_def.add()
        
        self.buffers = {}
        
        # allocate buffers
        for i, info in self.AUs.items():
            self.buffers[i] = scn.Buffer().read(info.path)

    @bundle
    def start(self):
        self.old_levels = {}
        
        # initialize old range levels
        for i in self.AUs.keys():
            self.old_levels[i] = 0
    
    @bundle
    def stop(self):
        # synths die out alone
        pass
    
    @bundle
    def _process(self, row):
        
        for i, info in self.AUs.items():
            intensity = row[info.label]
            cur_range_level = self.map_intensity(intensity, self.old_levels[i])
            
            if cur_range_level != self.old_levels[i] and cur_range_level >= 1:
                db = scn.linlin(cur_range_level, 1, 5, -40, 0, "minmax")
                amp = scn.dbamp(db)
            
                if cur_range_level > self.old_levels[i]:
                    scn.Synth(
                        "playbuf_bend",
                        {
                            "bufnum": self.buffers[i].bufnum,
                            "amp": self.amp * amp,
                            "pan": 1 if self.pan else 0,
                            "breakTime": info.break_time,
                            "rateFinal": info.rate_up
                        }
                    )
                else:
                    scn.Synth(
                        "playbuf_bend",
                        {
                            "bufnum": self.buffers[i].bufnum,
                            "amp": self.amp * amp,
                            "pan": -1 if self.pan else 0,
                            "breakTime": info.break_time,
                            "rateFinal": info.rate_down
                        }
                    )
                    
            # update old_range_level
            self.old_levels[i] = cur_range_level

    def map_intensity(self, intensity, old_level):
        cur_level = int(intensity)
        
        if cur_level == old_level:
            return cur_level        
        elif cur_level > old_level:
            return cur_level if intensity > cur_level + self.bounds[1] else old_level
        elif cur_level < old_level:
            return cur_level if intensity < old_level + self.bounds[0] else old_level
            
    def free(self):
        # deallocate buffers
        for buf in self.buffers.values():
            buf.free()

In [ ]:
class NoisyHead(ps.Sonification):
    
    # parameters of the sonification
    amp = ps.FloatSliderParameter(0, 1)
    base_tone = ps.MidiSliderParameter()
    log_mapping = ps.CheckboxParameter()
    
    # max expected values for each rotation
    rx_bound = ps.FloatSliderParameter(pi/2 / 10, pi/2)
    ry_bound = ps.FloatSliderParameter(pi/2 / 10, pi/2)
    rz_bound = ps.FloatSliderParameter(pi/2 / 10, pi/2)
    
    synth_def = scn.SynthDef(
        "bpf_noise",
        r"""{ | amp=1, pan=0, lg=0.5, freq=440, rq=0.2 |
            var sig;
            sig = PinkNoise.ar(amp);
            sig = BPF.ar(
                sig,
                freq.lag(lg),
                rq.lag(lg),
                // when a bandpass filter narrows, the amplitude decreases: this will balance it
                1/rq.sqrt.lag(lg)
            );
            Out.ar(0, Pan2.ar(sig, pan));
        }"""
    )
    
    def init_parameters(self):
        self.amp = 0.3
        self.base_tone = 69
        self.log_mapping = False
        
        self.rx_bound = pi/2
        self.ry_bound = pi/2
        self.rz_bound = pi/4
    
    @bundle
    def init_server(self):
        self.synth_def.add()

    @bundle
    def start(self):
        self.synth = scn.Synth("bpf_noise", {"amp": 0, "lg": 0.015})
    
    @bundle
    def _process(self, row):
        
        rx, ry, rz = row[['pose_Rx', 'pose_Ry', 'pose_Rz']]
        
        # use log() - 1 to map
        if self.log_mapping:
            rx_log = log2(scn.linlin(abs(rx), 0, self.rx_bound, 1, 2, 'minmax'))
            rx_midi = scn.linlin(rx_log, 0, 1, 0, 12, 'minmax')
            sign = 1 if rx >= 0 else -1
            pitch = self.base_tone + sign * rx_midi
            
            ry_exp = scn.linlin(abs(ry), 0, self.ry_bound, 1, 2, 'minmax')
            sign = 1 if ry >= 0 else -1
            pan = sign * log2(ry_exp)
                        
            q =   scn.linlin(rz, -self.rz_bound, +self.rz_bound, 1, 100, 'minmax')
            
        else:
            pitch = scn.linlin(rx, -self.rx_bound, +self.rx_bound, self.base_tone+12, self.base_tone-12, "minmax")
            pan =   scn.linlin(ry, -self.ry_bound, +self.ry_bound, +1, -1, "minmax")
            # linexp mapping to quality
            q =   exp(scn.linlin(rz, -self.rz_bound, +self.rz_bound, log2(1), log2(100), 'minmax'))
        
        self.synth.set(
            "amp", self.amp,
            "freq", scn.midicps(pitch),
            "pan", pan,
            "rq", 1/q
        )

In [ ]:
class SilentGaze(ps.Sonification):
    
    # parameters of the sonification
    amp = ps.FloatSliderParameter(0, 1, 0.01)
    
    gx_silence_thrashold = ps.FloatSliderParameter(0, pi/2, 0.01)
    gy_silence_thrashold = ps.FloatSliderParameter(0, pi/2, 0.01)
    
    freq = ps.FreqSliderParameter()
    
    def init_parameters(self):
        self.amp = 0.1
        self.gx_silence_thrashold = pi/2 / 10
        self.gy_silence_thrashold = pi/2 / 10
        
        self.freq = 50
    
    @bundle
    def init_server(self):
        scn.SynthDef.load("/home/michele/Desktop/Thesis/tools/sc3nb/src/sc3nb/resources/synthdefs/s2.scsyndef")

    @bundle
    def start(self):
        self.synth = scn.Synth("s2", {"amp": 0, "lg": 0.015})
    
    @bundle
    def _process(self, row):
        
        gx, gy = row[['gaze_angle_x', 'gaze_angle_y']]
        
        # linear mapping
        amp = max(
            # clips values under silence thrashold
            scn.linlin(abs(gx), self.gx_silence_thrashold, pi/2, 0, 1, "minmax"),
            scn.linlin(abs(gy), self.gy_silence_thrashold, pi/2, 0, 1, "minmax")
        )
        
        pan =   scn.linlin(gx, -pi/2, +pi/2, -1, +1)
        pitch = scn.linlin(gy, -pi/2, +pi/2, 69+12, 69-12)
        
        self.synth.set(
            "amp", self.amp * amp,
            "freq", scn.midicps(pitch),
            "pan", pan
        )

In [ ]:
class DustSmile(ps.Sonification):
    
    # parameters of the sonification
    amp = ps.FloatSliderParameter(0, 1)
    base_tone = ps.MidiSliderParameter()
    
    synth_def = scn.SynthDef(
        "discrete_rev",
        r"""{ | amp=1, freq=440, density=2, mix=0.5, room=0.5, damp=0.2, lg=0.1 |
            var trig, sig, env;
            sig = SinOsc.ar(freq);
            // transform signal into short blips
            trig = Dust.kr(density);
            env = EnvGen.kr(Env.perc(0.001, 0.05), trig);
            sig = sig * env;
            sig = FreeVerb.ar(sig, mix.lag(lg), room.lag(lg), damp.lag(lg), amp.lag(lg));
            Out.ar(0, sig!2);
        }"""
    )
    
    def init_parameters(self):
        self.amp = 0.3
        self.base_tone = 69
    
    @bundle
    def init_server(self):
        self.synth_def.add()

    @bundle
    def start(self):
        self.synth = scn.Synth("discrete_rev", {"amp": 0})
    
    @bundle
    def _process(self, row):
        # intensities
        au06, au12, au15, au17 = row[['AU06_r', 'AU12_r', 'AU15_r', 'AU17_r']]

        if au12 > 1 and au15 > 1:
            # this should not happen usually
            return
        
        if au12 > 1:
            pitch = scn.linlin(au12, 1, 5, self.base_tone, self.base_tone+12, 'minmax')
            mix = scn.linlin(au06, 1, 3, 0.2, 1, 'minmax')
            density = scn.linlin(max(au12, au06), 1, 5, 5, 30, 'minmax')
        elif au15 > 1:
            pitch = scn.linlin(au15, 1, 5, self.base_tone, self.base_tone-12, 'minmax')
            mix = scn.linlin(au17, 1, 3, 0.2, 1, 'minmax')
            density = scn.linlin(max(au15, au17), 1, 5, 5, 30, 'minmax')
        else:
            pitch = 69
            mix = 0
            density = 0
        
        self.synth.set(
            "amp", self.amp,
            "freq", scn.midicps(pitch),
            "mix", mix,
            "density", density,
        )

## Tests

### Sonification
This is a very simple sonification of AU04 (Brow Lowerer). The intensity of AU04 is used here to modulate both the amplitude and the frequency of a continuous synth. As continuous synth, the default synth of sc3nb s2 is used (we will have to instruct the server to load it).

* The intensity range \[0,1\] is mapped into the amplitude range \[0,0.3\], where 0.3 will be the maximum amplitude of the sound. The sonification has a parameter amp that can be used to scale this range.
* The intensity range \[0,5\] is mapped into the midi range \[69,81\]

In [ ]:
son = AU04ContinuousSonification()
son

#### Online

In [ ]:
rtdp = ps.RTDataPlayer(openface_stream, son)

In [ ]:
display(son)
rtdp

#### Offline

In [ ]:
dp = ps.DataPlayer(son).load('log.csv')

display(son)
display(dp)

In [ ]:
dp.export("score.wav", header_format="WAV")

### GroupSonification

In [ ]:
son = ps.GroupSonification([DirectionalPercussive(), DropBlink(), NoisyHead(), SilentGaze()])
son

In [ ]:
rtdp = ps.RTDataPlayer(openface_stream, son)
rtdp

In [ ]:
dp = ps.DataPlayer(son).load('log.csv')
dp

In [ ]:
dp.export("score.wav", header_format="WAV")

### Feature Display

In [ ]:
feature_display = ps.RTFeatureDisplay(['AU04_r', 'AU12_r'], 100).show()

In [ ]:
son = AU04ContinuousSonification()
son

In [ ]:
dp = ps.DataPlayer(son, feature_display=feature_display).load(df)
dp

### VideoPlayers

In [ ]:
vp = ps.VideoPlayer('/home/michele/Desktop/Thesis/media/files/phone.avi', fps=30)

In [ ]:
dp = ps.DataPlayer(son, video_player=vp).load(df)

display(son)
display(dp)

In [ ]:
vp.quit()

With long videos.

In [ ]:
vp = ps.VideoPlayer('long/movie', fps=25)

In [ ]:
import time

for i in range(10000):
    vp.seek(i)
    time.sleep(1 / 23.98)

In [ ]:
vp.seek_time(4001)

In [ ]:
vp.quit()

### Preprocessors

In [ ]:
avg_features = df.filter(regex='(AU.{2}_r)|(pose_R.)|(gaze_angle_.)').columns.to_list()
# avg_features

In [ ]:
class MovingAverage(ps.Preprocessor):

    def __init__(self):
        self.features = None
        self.avg_features = avg_features
        
        self.window_size = 5
        self.window = pd.DataFrame()
    
    def preprocess(self, row):        
        if self.features is None:
            self.features = row.index.to_list()
            
        self.window = self.window.append(row[self.avg_features])
    
        if self.window.shape[0] > self.window_size:
            self.window = self.window.drop(self.window.index[0])

        mean_series = self.window.mean()

        # write values into row
        for label in self.avg_features:
            row[label] = mean_series[label]

In [ ]:
from panson import streams

FIFO_PATH = os.path.join(FILE_DIR, 'pipe.csv')

openface_stream_avg = streams.CsvFifo('openface', args=(FIFO_PATH,), preprocessor=MovingAverage)\
        .add_open_hook(feature_extraction_online)\
        .add_close_hook(kill_feature_extraction_online)\
        .test()

In [ ]:
feature_display = ps.LiveFeatureDisplay(['AU04_r', 'AU12_r'], 100)

In [ ]:
rtdp = ps.RTDataPlayer(openface_stream, son, feature_display=feature_display)

In [ ]:
rtdp = ps.RTDataPlayer(openface_stream_avg, son, feature_display=feature_display)

In [ ]:
feature_display.show()
rtdp

### Streams

### Multi-stream

In [ ]:
class NoisyHeadMulti(ps.Sonification):
    
    # parameters of the sonification
    amp = ps.FloatSliderParameter(0, 1)
    base_tone = ps.MidiSliderParameter()
    log_mapping = ps.CheckboxParameter()
    
    # max expected values for each rotation
    rx_bound = ps.FloatSliderParameter(pi/2 / 10, pi/2)
    ry_bound = ps.FloatSliderParameter(pi/2 / 10, pi/2)
    rz_bound = ps.FloatSliderParameter(pi/2 / 10, pi/2)
    
    synth_def = scn.SynthDef(
        "bpf_noise",
        r"""{ | amp=1, pan=0, lg=0.5, freq=440, rq=0.2 |
            var sig;
            sig = PinkNoise.ar(amp);
            sig = BPF.ar(
                sig,
                freq.lag(lg),
                rq.lag(lg),
                // when a bandpass filter narrows, the amplitude decreases: this will balance it
                1/rq.sqrt.lag(lg)
            );
            Out.ar(0, Pan2.ar(sig, pan));
        }"""
    )
    
    def init_parameters(self):
        self.amp = 0.3
        self.base_tone = 69
        self.log_mapping = False
        
        self.rx_bound = pi/2
        self.ry_bound = pi/2
        self.rz_bound = pi/4
    
    @bundle
    def init_server(self):
        self.synth_def.add()

    @bundle
    def start(self):
        self.synth = scn.Synth("bpf_noise", {"amp": 0, "lg": 0.015})
    
    @bundle
    def _process(self, row):
        
        # use sin value as amplitude modulator
        amp_mod = scn.linlin(row['value'], -1, 1, 0, 1, 'minmax')
        
        rx, ry, rz = row[['pose_Rx', 'pose_Ry', 'pose_Rz']]
        
        # use log() - 1 to map
        if self.log_mapping:
            rx_log = log2(scn.linlin(abs(rx), 0, self.rx_bound, 1, 2, 'minmax'))
            rx_midi = scn.linlin(rx_log, 0, 1, 0, 12, 'minmax')
            sign = 1 if rx >= 0 else -1
            pitch = self.base_tone + sign * rx_midi
            
            ry_exp = scn.linlin(abs(ry), 0, self.ry_bound, 1, 2, 'minmax')
            sign = 1 if ry >= 0 else -1
            pan = sign * log2(ry_exp)
                        
            q =   scn.linlin(rz, -self.rz_bound, +self.rz_bound, 1, 100, 'minmax')
            
        else:
            pitch = scn.linlin(rx, -self.rx_bound, +self.rx_bound, self.base_tone+12, self.base_tone-12, "minmax")
            pan =   scn.linlin(ry, -self.ry_bound, +self.ry_bound, +1, -1, "minmax")
            # linexp mapping to quality
            q =   exp(scn.linlin(rz, -self.rz_bound, +self.rz_bound, log2(1), log2(100), 'minmax'))
        
        self.synth.set(
            "amp", self.amp * amp_mod,
            "freq", scn.midicps(pitch),
            "pan", pan,
            "rq", 1/q
        )

In [ ]:
son = NoisyHeadMulti()
son

In [ ]:
sin = streams.DummySin('sin', kwargs={'fps': 20, 'timestamps': False}).test()

In [ ]:
import math, time

class DummySinErr(ps.Stream):
    """Crashes after 10 seconds."""

    @staticmethod
    def datagen(fps=30, amp=1, timestamps=True):
        """Yields sinusoidal values varying with time."""

        header = ['sinerr_value']
        if timestamps:
            # head insert
            header.insert(0, 'timestamp')

        yield np.array(header)

        t0 = time.time()

        while True:
            t = time.time() - t0
            
            if t > 10:
                a = 1 / 0
            
            value = math.sin(t) * amp
            data = [value]
            if timestamps:
                data.insert(0, t)

            yield np.array(data)

            # TODO: improve timing
            time.sleep(1 / fps)

In [ ]:
sinerr = DummySinErr('sinerr', kwargs={'fps': 20, 'timestamps': False}).test()

#### Multi-threading

In [ ]:
rtdpmt = ps.RTDataPlayerMT([openface_stream, sin], son, fps=30)

In [ ]:
rtdpmt

In [ ]:
dp = ps.DataPlayer(son).load('log.csv')
dp

#### Multi-processing

In [ ]:
rtdpmp = ps.RTDataPlayerMP([openface_stream, sin], son, fps=30)

In [ ]:
rtdpmp

In [ ]:
dp = ps.DataPlayer(son).load('log.csv')
dp

In [ ]:
rtdpmp = ps.RTDataPlayerMP([openface_stream, sin, sinerr], son, fps=30)

In [ ]:
rtdpmp